In [7]:
import pkg.chat_model.qwen as qwen
from pkg.tools import calc

tool_kits = [calc.add]

qwen.chat_model

TypeError: tool() got an unexpected keyword argument 'description'

In [ ]:
from langchain.agents import initialize_agent,AgentType

agent = initialize_agent(
    tools=tool_kits,
    llm=qwen.chat_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

from langchain.prompts import ChatPromptTemplate

system_prompt_template = "您是一个{role}，擅长回答{domain}领域的问题。"
user_question = "{question}"
chat_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt_template),
        ("human", user_question)
    ]
)
messages = chat_prompt.format_messages(
    role="计算",
    domain="使用工具进行数学计算",
    question="100+100=？"
)

resp = agent.invoke(messages)

/Users/7yue/Workspace/AGI/learn-agents/agent-by-imooc/.venv/lib/python3.13/site-packages/langchain/agents/json_chat/base.py:22: SyntaxWarning: invalid escape sequence '\ '
  """Create an agent that uses JSON to format its logic, build for Chat Models.


NameError: name 'tool_kits' is not defined

## 格式化输出

In [17]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Summary(BaseModel):
    result: str = Field(description="计算结果")
    args: str = Field(description="计算参数")

parser = JsonOutputParser(pydantic_object=Summary)
format_instructions = parser.get_format_instructions()
print(format_instructions)

messages = chat_prompt.format_messages(
    role="计算",
    domain="使用工具进行数学计算",
    question=f"""
请阅读下面的问题，并返回一个严格的 JSON 对象，不要使用 Markdown 代码块包裹！
格式要求：
{format_instructions}

问题：
100+100=？
"""
)

resp = agent.invoke(messages)
print(resp["output"])

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"result": {"description": "计算结果", "title": "Result", "type": "string"}, "args": {"description": "计算参数", "title": "Args", "type": "string"}}, "required": ["result", "args"]}
```


> Entering new AgentExecutor chain...
Thought: 我需要计算 100 + 100 的结果，并按照指定的 JSON 格式返回。
Action:
```json
{
  "action": "add",
  "action_input": {
    "a": 100,
    "b": 100
  }
}
```
Observation: 200


> Finished chain.
200


## OutputParser

- StrOutputParser：直接提取模型返回的原始文本，不做任何结构化处理。
- CommaSeparatedListOutputParser：将逗号分隔的文本转换为Python列表。如：Output: "1,2,3,4" -> ["1", "2", "3", "4"]
- ~~BooleanOutputParser~~：将文本转换为Python布尔值。如：Output: "True" -> True
- SimpleJsonOutputParser：将文本简单处理后转换为JSON格式，通常用于模型已经正确输出JSON的情况。
- ~~DatetimeOutputParser~~：将文本解析为标准日期时间格式（"%Y-%m-%dT%H:%M:%S.%fZ"）
- ~~EnumOutputParser~~：将文本解析为枚举值，确保输出符合预定义的枚举类型。

Pydantic 解析器
> Pydantic解析器通过Pydantic模型定义复杂结构，提供更严格的验证和数据校验
- PydanticOutputParser：将模型返回的文本解析为Pydantic模型实例，确保输出符合模型定义的结构。
- PydanticToolsParser：专门处理OpenAI工具调用中的Pydantic对象，将工具调用参数解析为Pydantic模型。

函数调用解析器
- OutputFunctionsParser：解析模型返回的函数调用参数，通常返回JSON格式的函数参数
- JsonOutputFunctionsParser：提取函数调用中的JSON参数。


In [27]:
from langchain_core.output_parsers import StrOutputParser,CommaSeparatedListOutputParser,NumberedListOutputParser,SimpleJsonOutputParser,

v = StrOutputParser().parse(resp["output"])
print(type(v), v)

v = CommaSeparatedListOutputParser().parse("1,2,3,4")
print( type(v), v)

v= NumberedListOutputParser().parse("1. a\n2. v\n3. v")
print(type(v), v)

# SimpleJsonOutputParser().parse(resp["output"])


# v = DateTimeOutputParser().parse("2023-10-10T10:00:00.000Z")
# print(type(v), v)


ImportError: cannot import name 'DateTimeOutputParser' from 'langchain_core.output_parsers' (/Users/boohee/Workspace/llms/learn-agents/.venv/lib/python3.13/site-packages/langchain_core/output_parsers/__init__.py)